### Задание

Ознакомиться со структурой датасета, в котором хранятся сведения о результатах сходов/крушений подвижного состава вне стрелочных переводов по причине неисправности подвижного состава.

Загрузить указанный датасет в любой математический пакет по выбору студента.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

df = pd.read_excel("WO.xlsx")
df.head()

,date,length,commonlength,maxder,dcar,speed,weight,load,curve,profile,type
0,2013-01-03,68,70,61,1,6,4875,0.705669,0.005000,0.0000,2
1,2013-01-03,43,45,2,1,30,3955,0.999663,0.005000,NaN,2
2,2013-01-10,70,72,65,4,10,1658,0.009938,NaN,NaN,2
3,2013-01-10,56,58,44,10,29,2651,0.352743,0.003333,-0.0164,2
4,2013-01-12,68,71,23,1,5,2522,0.204177,0.003333,NaN,2


Используя геометрическую регрессию, построить не менее 12 зависимостей среднего количества подвижных единиц в сходе с рельсов от различных факторов движения. Для каждой построенной зависимости привести значения скорректированного коэффициента детерминации, средней абсолютной погрешности, средней относительной погрешности, используя два прогноза (в виде среднего и в виде значения с максимальной вероятностью). Привести значение AIC, отношения правдоподобия (при сравнении с тривиальной моделью, содержащей только константу).

Определить наилучшую из построенных зависимостей и объяснить, почему она, на взгляд студента, является наилучшей.

### Решение

Напишем метод, который будет применять метод геометрической регрессии из библиотеки `statsmodels.api.GLM`. Метод заключается заключается максимизации функции $\overline{L}\left( x_1, \dots, x_n, \dots, \theta_1, \dots \theta_p, y_1, \dots, y_n \right)$
$$
P(Y = y|X = x, \theta_1, \dots, \theta_p) = (1 - g(x, \theta_1, \dots, \theta_p))^{y} \cdot g(x, \theta_1, \dots, \theta_p)
$$
$$
L\left( x_1, \dots, x_n, \theta_1, \dots \theta_p, y_1, \dots, y_n \right) = \prod\limits_{k=1}^n (1 - g(x_k, \theta_1, \dots, \theta_p))^{y_k} \cdot g(x_k, \theta_1, \dots, \theta_p) $$
$$
\overline{L}\left( x_1, \dots, x_n, \theta_1, \dots \theta_p, y_1, \dots, y_n \right)=\sum\limits_{k=1}^{n} \left( y_k \ln{\left( 1 - g(x_k, \theta_1, \dots, \theta_p) \right)}  \right) + \sum\limits_{k=1}^{n} \ln{g(x_k, \theta_1, \dots, \theta_p)} 
$$

где $y_k, k=1,\dots,n$ - набор скалярных экспериментальных данных, интересующий нас столбец таблицы `dcar` - 1, $x_k$ - набор векторных экспериментальных данных и предполагается, что $y$ зависит от $x$, $\theta_1, \dots, \theta_p$ - параметры системы, $g(x, \theta_1, \dots, \theta_p) = \frac{\exp(\cdot)}{1+\exp(\cdot)}$ - функция, которая определяется вектором неизвестных параметров $\theta_1, \dots, \theta_p$.

In [2]:
def solve_geometric(df, target_col='dcar', feature_cols=None):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col != target_col]

    y_original = df[target_col]
    y_shifted = y_original - 1
    
    X = -1 * df[feature_cols]
    X_sm = sm.add_constant(X, has_constant='add')

    n = X.shape[0]
    p = X.shape[1]

    try:
        nb_model = sm.GLM(y_shifted, X_sm, family=sm.families.NegativeBinomial())
        nb_model_sm = nb_model.fit()
        if not nb_model_sm.converged:
            print("Внимание: Модель Geometric может не сойтись. Попытка с другим методом...")
            try:
                nb_model_sm = nb_model.fit(method='newton')
                if not nb_model_sm.converged:
                    print("Ошибка: Модель не сошлась и с методом Newton.")
                    return None
            except Exception as e_newton:
                print(f"Ошибка при попытке с методом Newton: {e_newton}")
                return None

    except Exception as e:
        print(f"Ошибка при построении геометрической модели: {e}")
        return None
    y_pred_shifted_mean = nb_model_sm.predict(X_sm)
    y_pred_mean = y_pred_shifted_mean + 1
    alpha = nb_model_sm.scale

    if alpha > 1:
        y_pred_shifted_mode = np.floor(y_pred_shifted_mean * (alpha - 1) / alpha)
        y_pred_shifted_mode = np.maximum(0, y_pred_shifted_mode)
    else:
        y_pred_shifted_mode = np.zeros_like(y_pred_shifted_mean)
    
    y_pred_mode = y_pred_shifted_mode + 1
    
    y_pred_mode = y_pred_mode.astype(int)

    null_model = sm.GLM(y_shifted, sm.add_constant(np.ones(n)),
                      family=sm.families.NegativeBinomial()).fit()
                      
    llf = nb_model_sm.llf
    llnull = null_model.llf
    
    y_np = y_original.to_numpy()
    r2_mean = 1 - sum([(y_np[i] - y_pred_mean[i])**2 for i in range(len(y_np))])/sum([(y_np[i] - np.mean(y_np))**2 for i in range(len(y_np))])
    adj_r2_mean = 1 - (1-r2_mean)*(n-1)/(n-p-1)

    r2_mode = 1 - sum([(y_np[i] - y_pred_mode[i])**2 for i in range(len(y_np))])/sum([(y_np[i] - np.mean(y_np))**2 for i in range(len(y_np))])
    adj_r2_mode = 1 - (1-r2_mode)*(n-1)/(n-p-1)

    mae_mean = mean_absolute_error(y_original, y_pred_mean)
    mae_mode = mean_absolute_error(y_original, y_pred_mode)

    valid_indices = y_original != 0
    if np.any(valid_indices):
        mre_mean = np.mean(np.abs((y_original[valid_indices] - y_pred_mean[valid_indices]) / y_original[valid_indices]))
        mre_mode = np.mean(np.abs((y_original[valid_indices] - y_pred_mode[valid_indices]) / y_original[valid_indices]))
    else:
        mre_mean = 0
        mre_mode = 0

    aic = nb_model_sm.aic

    likelihood_ratio = -2 * (llnull - llf)
    

    params = nb_model_sm.params
    intercept = params.get('const', np.nan)
    coefs = params.drop('const', errors='ignore')

    estimated_alpha = nb_model_sm.scale

    formula = "1 + 1/exp("
    formula_parts = []
    for feature in feature_cols:
        if feature in coefs.index:
            # Инвертируем обратно коэффициенты, т.к. мы инвертировали признаки
            formula_parts.append(f"{-coefs[feature]:.3f}*{feature}")
    formula += " + ".join(formula_parts)
    if not pd.isna(intercept):
        formula += f" + {-intercept:.3f}"  # Инвертируем обратно и константу
    formula += ")"
    
    alt_formula = "1 + exp(-("
    alt_formula_parts = []
    for feature in feature_cols:
        if feature in coefs.index:
            alt_formula_parts.append(f"{-coefs[feature]:.3f}*{feature}")
    alt_formula += " + ".join(alt_formula_parts)
    if not pd.isna(intercept):
        alt_formula += f" + {-intercept:.3f}"
    alt_formula += "))"

    return {
        'params': params[1:],
        'intercept': -intercept,
        'estimated_alpha': estimated_alpha,
        'formula': formula,
        'alt_formula': alt_formula, 
        'y': y_original.to_numpy(), 
        'y_pred_mean': np.array(y_pred_mean),  
        'y_pred_mode': np.array(y_pred_mode),  
        'r2_mean': r2_mean,        
        'adj_r2_mean': adj_r2_mean,
        'r2_mode': r2_mode,        
        'adj_r2_mode': adj_r2_mode,
        'mae_mean': mae_mean,
        'mae_mode': mae_mode,
        'mre_mean': mre_mean,
        'mre_mode': mre_mode,
        'aic': aic,
        'likelihood_ratio': likelihood_ratio
    }


In [3]:
data = {
    'Зависимость': [],
    'Прогноз': [],
    'R^2_adj': [],
    'Δ': [],
    '𝛿': [],
    'AIC': [],
    'Отношение правдоподобия': []
}

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

result_table = pd.DataFrame(data)

In [4]:
# Тест 1

dd = df[['length', 'dcar']].copy().dropna()
res_1 = solve_geometric(dd)
ans_1 = "dcar = "
for i in range(len(res_1['params'])):
    ans_1 += f"{res_1['params'].values[i]:.4f}*{res_1['params'].keys()[i]} + "
ans_1 += str(format(res_1['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_1, "Среднее", res_1['adj_r2_mean'], res_1['mae_mean'], res_1['mre_mean'], res_1['aic'], res_1['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_1, "Макс. вер-ть", res_1['adj_r2_mode'], res_1['mae_mode'], res_1['mre_mode'], res_1['aic'], res_1['likelihood_ratio']]
res_1

{'params': length   -0.002862
 dtype: float64,
 'intercept': -0.6404528261747815,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(0.003*length + -0.640)',
 'alt_formula': '1 + exp(-(0.003*length + -0.640))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([3.3049298 , 3.14578845, 3.31815978, 3.22712052, 3.3049298 ,
        3.24632306, 3.31815978, 3.25276063, 3.29177533, 3.52597081,
        3.31815978, 3.39914972, 3.28522627, 3.32480323, 3.31153533,
        3.25276063, 3.32480323, 3.22712052, 3.28522627, 3.14578845,
        3.31815978, 3.49722112, 2.9580162 , 3.37184346, 3.28522627,
        3.07335143, 3.18295054, 3.31815978, 3.54775008, 3.22075622,
        3.3049298 , 3.15193791, 3.17049221, 3.16428974, 3.26569117,
        3.29834315, 3.07929329, 3.25276063, 3.49722112, 3.25276063]),
 'y_pred_mode': array([1, 1

In [5]:
# Тест 2

dd = df[['maxder', 'dcar']].copy().dropna()
res_2 = solve_geometric(dd)
ans_2 = "dcar = "
for i in range(len(res_2['params'])):
    ans_2 += f"{res_2['params'].values[i]:.4f}*{res_2['params'].keys()[i]} + "
ans_2 += str(format(res_2['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_2, "Среднее", res_2['adj_r2_mean'], res_2['mae_mean'], res_2['mre_mean'], res_2['aic'], res_2['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_2, "Макс. вер-ть", res_2['adj_r2_mode'], res_2['mae_mode'], res_2['mre_mode'], res_2['aic'], res_2['likelihood_ratio']]
res_2

{'params': maxder   -0.02023
 dtype: float64,
 'intercept': -0.02144300049172307,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(0.020*maxder + -0.021)',
 'alt_formula': '1 + exp(-(0.020*maxder + -0.021))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([4.50945165, 2.06385869, 4.80523951, 3.48818488, 2.62698405,
        4.37029485, 2.80016251, 2.25074683, 2.91279647, 8.88254755,
        2.04255328, 4.108316  , 2.15352411, 2.76411149, 2.69416093,
        3.75303075, 2.06385869, 3.24881759, 2.72878245, 3.48818488,
        5.56512998, 2.53117999, 2.32900453, 5.56512998, 2.53117999,
        2.95188598, 2.08559948, 4.58117008, 2.04255328, 2.83695025,
        4.108316  , 3.48818488, 3.69789709, 3.0740131 , 2.08559948,
        2.06385869, 2.30238917, 3.75303075, 3.98506503, 2.87448979]),
 'y_pred_mode': array([1, 1

In [6]:
# Тест 3

dd = df[['weight', 'dcar']].copy().dropna()
res_3 = solve_geometric(dd)
ans_3 = "dcar = "
for i in range(len(res_3['params'])):
    ans_3 += f"{res_3['params'].values[i]:.4f}*{res_3['params'].keys()[i]} + "
ans_3 += str(format(res_3['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_3, "Среднее", res_3['adj_r2_mean'], res_3['mae_mean'], res_3['mre_mean'], res_3['aic'], res_3['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_3, "Макс. вер-ть", res_3['adj_r2_mode'], res_3['mae_mode'], res_3['mre_mode'], res_3['aic'], res_3['likelihood_ratio']]
res_3

{'params': weight    0.000052
 dtype: float64,
 'intercept': -1.0228339347317374,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.000*weight + -1.023)',
 'alt_formula': '1 + exp(-(-0.000*weight + -1.023))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([3.15500662, 3.26126513, 3.55000197, 3.42091221, 3.43730561,
        3.32180289, 3.55013537, 3.40148612, 3.04601991, 3.45702145,
        3.23118218, 3.06689129, 3.03353455, 3.19150212, 3.08950543,
        3.10784092, 3.53842204, 3.56404811, 3.31949616, 3.42217906,
        3.55040221, 2.73870669, 3.70119418, 2.92695753, 3.032152  ,
        3.66247135, 3.19058511, 2.93372369, 3.10189454, 3.12834058,
        3.13079161, 3.32496319, 3.23527135, 3.3297117 , 3.05912048,
        3.00469752, 3.67559682, 3.35163088, 3.39646594, 3.24089154]),
 'y_pred_mode': array([1,

In [7]:
# Тест 4

dd = df[['speed', 'dcar']].copy().dropna()
res_4 = solve_geometric(dd)
ans_4 = "dcar = "
for i in range(len(res_4['params'])):
    ans_4 += f"{res_4['params'].values[i]:.4f}*{res_4['params'].keys()[i]} + "
ans_4 += str(format(res_4['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_4, "Среднее", res_4['adj_r2_mean'], res_4['mae_mean'], res_4['mre_mean'], res_4['aic'], res_4['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_4, "Макс. вер-ть", res_4['adj_r2_mode'], res_4['mae_mode'], res_4['mre_mode'], res_4['aic'], res_4['likelihood_ratio']]
res_4

{'params': speed   -0.0329
 dtype: float64,
 'intercept': 0.07692448718046979,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(0.033*speed + 0.077)',
 'alt_formula': '1 + exp(-(0.033*speed + 0.077))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.12802982,  3.48447383,  2.28668693,  3.4040658 ,  2.09152203,
         3.1076284 ,  9.9633308 ,  2.28668693,  2.73007759,  2.97341256,
         2.05619578,  2.56747762,  9.39253713,  2.84774372,  1.9889362 ,
         2.24504432,  2.61990456,  3.4040658 ,  2.28668693,  2.56747762,
        10.26312451,  3.17812158,  2.84774372,  2.42015949,  2.67408499,
         2.73007759,  2.32972234,  2.56747762,  4.12788635,  2.46765911,
         2.46765911,  2.46765911,  2.51674745,  2.73007759,  8.6037712 ,
         2.09152203,  2.61990456,  2.73007759,  2.51674745,  2.467659

In [8]:
# Тест 5

dd = df[['length', 'speed', 'dcar']].copy()
res_5 = solve_geometric(dd)
ans_5 = "dcar = "
for i in range(len(res_5['params'])):
    ans_5 += f"{res_5['params'].values[i]:.4f}*{res_5['params'].keys()[i]} + "
ans_5 += str(format(res_5['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_5, "Среднее", res_5['adj_r2_mean'], res_5['mae_mean'], res_5['mre_mean'], res_5['aic'], res_5['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_5, "Макс. вер-ть", res_5['adj_r2_mode'], res_5['mae_mode'], res_5['mre_mode'], res_5['aic'], res_5['likelihood_ratio']]
res_5

{'params': length    0.004237
 speed    -0.033502
 dtype: float64,
 'intercept': -0.17464368714240308,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.004*length + 0.034*speed + -0.175)',
 'alt_formula': '1 + exp(-(-0.004*length + 0.034*speed + -0.175))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.09144869,  3.71144128,  2.2374308 ,  3.48157086,  2.05548915,
         3.14292789,  9.93191272,  2.29099326,  2.70148494,  2.68444148,
         2.01210043,  2.43791947,  9.53192236,  2.78125331,  1.95052778,
         2.2484594 ,  2.5578566 ,  3.48157086,  2.26392833,  2.69631491,
        10.23621476,  2.89437435,  3.29691019,  2.32265669,  2.65241399,
         2.97350759,  2.39867525,  2.5129279 ,  3.65845146,  2.50773047,
         2.42692037,  2.57967255,  2.61285642,  2.85197116,  8.81482319,
         2.

In [9]:
# Тест 6

dd = df[['weight', 'maxder', 'dcar']].copy().dropna()
res_6 = solve_geometric(dd)
ans_6 = "dcar = "
for i in range(len(res_6['params'])):
    ans_6 += f"{res_6['params'].values[i]:.4f}*{res_6['params'].keys()[i]} + "
ans_6 += str(format(res_6['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_6, "Среднее", res_6['adj_r2_mean'], res_6['mae_mean'], res_6['mre_mean'], res_6['aic'], res_6['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_6, "Макс. вер-ть", res_6['adj_r2_mode'], res_6['mae_mode'], res_6['mre_mode'], res_6['aic'], res_6['likelihood_ratio']]
res_6

{'params': weight   -0.000046
 maxder   -0.021890
 dtype: float64,
 'intercept': 0.22266443097675356,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(0.000*weight + 0.022*maxder + 0.223)',
 'alt_formula': '1 + exp(-(0.000*weight + 0.022*maxder + 0.223))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([4.81155231, 2.00401242, 4.58526581, 3.37042445, 2.48798258,
        4.41562852, 2.59499338, 2.13421699, 3.06931973, 9.14734428,
        1.99397111, 4.46807708, 2.20368686, 2.78415479, 2.78120933,
        3.98894788, 1.90647504, 3.01949287, 2.66010676, 3.36932855,
        5.36536007, 2.87816384, 2.0916148 , 6.59273595, 2.63631275,
        2.67585627, 2.05544587, 5.28746064, 2.04782405, 2.91281065,
        4.37598483, 3.45668915, 3.77637908, 3.01376141, 2.11479253,
        2.11678597, 2.07700558, 3.7133442 , 3.912

In [10]:
# Тест 7

dd = df[['length', 'speed', 'maxder', 'dcar']].copy().dropna()
res_7 = solve_geometric(dd)
ans_7 = "dcar = "
for i in range(len(res_7['params'])):
    ans_7 += f"{res_7['params'].values[i]:.4f}*{res_7['params'].keys()[i]} + "
ans_7 += str(format(res_7['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_7, "Среднее", res_7['adj_r2_mean'], res_7['mae_mean'], res_7['mre_mean'], res_7['aic'], res_7['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_7, "Макс. вер-ть", res_7['adj_r2_mode'], res_7['mae_mode'], res_7['mre_mode'], res_7['aic'], res_7['likelihood_ratio']]
res_7

{'params': length    0.008410
 speed    -0.031709
 maxder   -0.018750
 dtype: float64,
 'intercept': 0.23879740951819894,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.008*length + 0.032*speed + 0.019*maxder + 0.239)',
 'alt_formula': '1 + exp(-(-0.008*length + 0.032*speed + 0.019*maxder + 0.239))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.68760808,  2.47453477,  3.03059228,  3.8145273 ,  1.80180574,
         4.20263814,  7.58911737,  1.78756395,  2.47676844,  5.67971316,
         1.50564854,  2.84073247,  5.26975325,  2.39965393,  1.75056687,
         2.58521825,  1.77154832,  3.56265894,  2.01932007,  3.07904157,
        17.11287834,  2.16564084,  2.78305767,  3.47174051,  2.17387411,
         3.01971623,  1.78204744,  3.32177612,  2.0908469 ,  2.33152071,
         2.94349107,  2.93494494,  3.

In [11]:
# Тест 8

dd = df[['length', 'speed', 'weight', 'dcar']].copy().dropna()
res_8 = solve_geometric(dd)
ans_8 = "dcar = "
for i in range(len(res_8['params'])):
    ans_8 += f"{res_8['params'].values[i]:.4f}*{res_8['params'].keys()[i]} + "
ans_8 += str(format(res_8['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_8, "Среднее", res_8['adj_r2_mean'], res_8['mae_mean'], res_8['mre_mean'], res_8['aic'], res_8['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_8, "Макс. вер-ть", res_8['adj_r2_mode'], res_8['mae_mode'], res_8['mre_mode'], res_8['aic'], res_8['likelihood_ratio']]
res_8

{'params': length    0.009103
 speed    -0.035801
 weight   -0.000068
 dtype: float64,
 'intercept': -0.15457647424701482,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.009*length + 0.036*speed + 0.000*weight + -0.155)',
 'alt_formula': '1 + exp(-(-0.009*length + 0.036*speed + 0.000*weight + -0.155))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.08675798,  4.0258859 ,  1.98861752,  3.37269238,  1.89292117,
         3.11290199,  9.1724286 ,  2.17104874,  2.88618102,  2.25777407,
         1.94937985,  2.44520238, 11.69859881,  2.7700234 ,  1.97158457,
         2.33952027,  2.26617255,  3.20132132,  2.17083609,  2.67576393,
         9.46915689,  3.28066919,  3.32634394,  2.47513483,  2.85286325,
         2.83945752,  2.51255557,  2.75840394,  3.47685524,  2.65566798,
         2.46867082,  2.63072286, 

In [12]:
# Тест 9

dd = df[['length', 'weight', 'speed', 'maxder', 'dcar']].copy().dropna()
res_9 = solve_geometric(dd)
ans_9 = "dcar = "
for i in range(len(res_9['params'])):
    ans_9 += f"{res_9['params'].values[i]:.4f}*{res_9['params'].keys()[i]} + "
ans_9 += str(format(res_9['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_9, "Среднее", res_9['adj_r2_mean'], res_9['mae_mean'], res_9['mre_mean'], res_9['aic'], res_9['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_9, "Макс. вер-ть", res_9['adj_r2_mode'], res_9['mae_mode'], res_9['mre_mode'], res_9['aic'], res_9['likelihood_ratio']]
res_9

{'params': length    0.029444
 weight   -0.000253
 speed    -0.036614
 maxder   -0.028714
 dtype: float64,
 'intercept': 0.4395239377359884,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.029*length + 0.000*weight + 0.037*speed + 0.029*maxder + 0.440)',
 'alt_formula': '1 + exp(-(-0.029*length + 0.000*weight + 0.037*speed + 0.029*maxder + 0.440))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 3.14263887,  2.56849397,  2.16318318,  3.47705354,  1.38266632,
         4.68768403,  4.4857175 ,  1.43010921,  2.98627633,  3.60398737,
         1.28342556,  3.10733222,  6.95883158,  2.17994625,  1.74199253,
         3.38642213,  1.24438798,  2.62565102,  1.6951443 ,  3.25090469,
        14.53991782,  2.69875464,  2.68132924,  6.14332545,  2.48601735,
         2.60453788,  1.78660431,  6.06107088,  1.47915235,  

In [13]:
# Тест 10

dd = df[['length', 'commonlength', 'maxder', 'speed', 'weight', 'load', 'dcar']].copy().dropna()
res_10 = solve_geometric(dd)
ans_10 = "dcar = "
for i in range(len(res_10['params'])):
    ans_10 += f"{res_10['params'].values[i]:.4f}*{res_10['params'].keys()[i]} + "
ans_10 += str(format(res_10['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_10, "Среднее", res_10['adj_r2_mean'], res_10['mae_mean'], res_10['mre_mean'], res_10['aic'], res_10['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_10, "Макс. вер-ть", res_10['adj_r2_mode'], res_10['mae_mode'], res_10['mre_mode'], res_10['aic'], res_10['likelihood_ratio']]
res_10

{'params': length          0.367174
 commonlength   -0.301146
 maxder         -0.033042
 speed          -0.035262
 weight         -0.000844
 load            2.797036
 dtype: float64,
 'intercept': -0.1621401448757428,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.367*length + 0.301*commonlength + 0.033*maxder + 0.035*speed + 0.001*weight + -2.797*load + -0.162)',
 'alt_formula': '1 + exp(-(-0.367*length + 0.301*commonlength + 0.033*maxder + 0.035*speed + 0.001*weight + -2.797*load + -0.162))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.90322102,  1.66498498,  2.01493119,  3.21391535,  1.39457144,
         4.23626106,  3.39270673,  1.32993287,  3.67821435,  2.32519795,
         1.19503159,  2.91342427,  4.61740277,  1.89227225,  1.57151539,
         2.75924056,  1.28752072,  4.00324163,  1.98433654

In [14]:
# Тест 11

dd = df[['length', 'commonlength', 'maxder', 'speed', 'weight', 'load', 'dcar']].copy().dropna()
dd['speed^2'] = dd['speed']**2
res_11 = solve_geometric(dd)
ans_11 = "dcar = "
for i in range(len(res_11['params'])):
    ans_11 += f"{res_11['params'].values[i]:.4f}*{res_11['params'].keys()[i]} + "
ans_11 += str(format(res_11['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_11, "Среднее", res_11['adj_r2_mean'], res_11['mae_mean'], res_11['mre_mean'], res_11['aic'], res_11['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_11, "Макс. вер-ть", res_11['adj_r2_mode'], res_11['mae_mode'], res_11['mre_mode'], res_11['aic'], res_11['likelihood_ratio']]
res_11

{'params': length          0.376774
 commonlength   -0.317873
 maxder         -0.032199
 speed          -0.080143
 weight         -0.000727
 load            2.289282
 speed^2         0.000574
 dtype: float64,
 'intercept': 0.6929314727998945,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.377*length + 0.318*commonlength + 0.032*maxder + 0.080*speed + 0.001*weight + -2.289*load + -0.001*speed^2 + 0.693)',
 'alt_formula': '1 + exp(-(-0.377*length + 0.318*commonlength + 0.032*maxder + 0.080*speed + 0.001*weight + -2.289*load + -0.001*speed^2 + 0.693))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.33584998,  1.94936057,  1.8513087 ,  3.77781011,  1.28591984,
         4.82917556,  3.01341792,  1.27956471,  3.9524547 ,  2.7210725 ,
         1.13413626,  2.8405974 ,  4.18236171,  2.00493707,  1.34954726,
 

In [15]:
# Тест 12

dd = df[['length', 'commonlength', 'maxder', 'speed', 'weight', 'load', 'dcar']].copy().dropna()
dd['weight*speed'] = dd['weight']*dd['speed']
res_12 = solve_geometric(dd)
ans_12 = "dcar = "
for i in range(len(res_12['params'])):
    ans_12 += f"{res_12['params'].values[i]:.4f}*{res_12['params'].keys()[i]} + "
ans_12 += str(format(res_12['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_12, "Среднее", res_12['adj_r2_mean'], res_12['mae_mean'], res_12['mre_mean'], res_12['aic'], res_12['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_12, "Макс. вер-ть", res_12['adj_r2_mode'], res_12['mae_mode'], res_12['mre_mode'], res_12['aic'], res_12['likelihood_ratio']]
res_12

{'params': length          0.426905
 commonlength   -0.360133
 maxder         -0.036288
 speed          -0.017638
 weight         -0.000739
 load            2.886661
 weight*speed   -0.000005
 dtype: float64,
 'intercept': -0.3740292529096848,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.427*length + 0.360*commonlength + 0.036*maxder + 0.018*speed + 0.001*weight + -2.887*load + 0.000*weight*speed + -0.374)',
 'alt_formula': '1 + exp(-(-0.427*length + 0.360*commonlength + 0.036*maxder + 0.018*speed + 0.001*weight + -2.887*load + 0.000*weight*speed + -0.374))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.7848725 ,  1.56638002,  2.26062059,  3.16669063,  1.43680917,
         4.37454606,  2.48985827,  1.31388571,  3.66951707,  2.60146749,
         1.15969567,  2.80969677,  6.0086142 ,  1.82456418,  1.

In [16]:
# Тест 13

dd = df[['length', 'commonlength', 'maxder', 'speed', 'weight', 'load', 'dcar']].copy().dropna()
dd['speed^2'] = dd['speed']**2
dd['weight*speed'] = dd['weight']*dd['speed']
res_13 = solve_geometric(dd)
ans_13 = "dcar = "
for i in range(len(res_13['params'])):
    ans_13 += f"{res_13['params'].values[i]:.4f}*{res_13['params'].keys()[i]} + "
ans_13 += str(format(res_13['intercept'], '.4f'))
result_table.loc[len(result_table)] = [ans_13, "Среднее", res_13['adj_r2_mean'], res_13['mae_mean'], res_13['mre_mean'], res_13['aic'], res_13['likelihood_ratio']]
result_table.loc[len(result_table)] = [ans_13, "Макс. вер-ть", res_13['adj_r2_mode'], res_13['mae_mode'], res_13['mre_mode'], res_13['aic'], res_13['likelihood_ratio']]
res_13

{'params': length          0.426264
 commonlength   -0.365695
 maxder         -0.035053
 speed          -0.059948
 weight         -0.000662
 load            2.439844
 speed^2         0.000492
 weight*speed   -0.000004
 dtype: float64,
 'intercept': 0.41736144828476884,
 'estimated_alpha': 1.0,
 'formula': '1 + 1/exp(-0.426*length + 0.366*commonlength + 0.035*maxder + 0.060*speed + 0.001*weight + -2.440*load + -0.000*speed^2 + 0.000*weight*speed + 0.417)',
 'alt_formula': '1 + exp(-(-0.426*length + 0.366*commonlength + 0.035*maxder + 0.060*speed + 0.001*weight + -2.440*load + -0.000*speed^2 + 0.000*weight*speed + 0.417))',
 'y': array([ 1,  1,  4, 10,  1,  4,  2,  1,  5,  2,  1,  3,  6,  3,  2,  1,  1,
         2,  3,  1, 22,  7,  3,  1,  3,  2,  3,  4,  1,  1,  4,  9,  2,  1,
         3,  1,  2,  6,  1,  1], dtype=int64),
 'y_pred_mean': array([ 2.3353553 ,  1.78646539,  2.03874733,  3.64319782,  1.32384351,
         4.86809517,  2.42227532,  1.27315262,  3.907696  ,  2.94201782,
     

In [17]:
styled_results = result_table.style \
    .format("{:.4f}", subset=["R^2_adj"]) \
    .format("{:.4f}", subset=["Δ"]) \
    .format("{:.4f}", subset=["𝛿"]) \
    .format("{:.4f}", subset=["AIC"]) \
    .format("{:.4f}", subset=["Отношение правдоподобия"]) \
    .highlight_max(color="grey", subset=["R^2_adj"])  

styled_results

,Зависимость,Прогноз,R^2_adj,Δ,𝛿,AIC,Отношение правдоподобия
0,dcar = -0.0029*length + -0.6405,Среднее,-0.0259,2.2421,1.0996,165.1542,0.0607
1,dcar = -0.0029*length + -0.6405,Макс. вер-ть,-0.4112,2.2750,0.4308,165.1542,0.0607
2,dcar = -0.0202*maxder + -0.0214,Среднее,0.0282,2.2594,1.0651,158.9430,6.2719
3,dcar = -0.0202*maxder + -0.0214,Макс. вер-ть,-0.4112,2.2750,0.4308,158.9430,6.2719
4,dcar = 0.0001*weight + -1.0228,Среднее,-0.0193,2.2769,1.1037,164.8666,0.3484
5,dcar = 0.0001*weight + -1.0228,Макс. вер-ть,-0.4112,2.2750,0.4308,164.8666,0.3484
6,dcar = -0.0329*speed + 0.0769,Среднее,0.2286,2.1690,0.9379,153.9302,11.2847
7,dcar = -0.0329*speed + 0.0769,Макс. вер-ть,-0.4112,2.2750,0.4308,153.9302,11.2847
8,dcar = 0.0042*length + -0.0335*speed + -0.1746,Среднее,0.2028,2.1893,0.9373,155.7952,11.4198
9,dcar = 0.0042*length + -0.0335*speed + -0.1746,Макс. вер-ть,-0.4494,2.2750,0.4308,155.7952,11.4198


### Заключение

Для построения прогноза количества подвижных единиц грузового состава, сошедших с рельсов в данной лабораторной работе был применен метод геометрической регрессии. Наилучшим образом показала себя модель с параметрами `length`, `commonlength`, `maxder`, `speed`, `weight` и `load` в случае прогноза в виде среднего. Скорректированный коэффициент детерминации $R^2_{adj}$ в этом случае оказался равен $0.5671$, значение AIC для этой модели одно из самых низких из рассмотренных, а отношение правдоподобия высокое, что тоже является плюсом. Недостатком этой модели является ее сложноть, то есть большое количество признаков, на такой маленькой выборке и с таким большим числом признаков есть риск переобучения модели. Все модели показали плохие результаты прогнозов в виде максимальной вероятности, что говорит о том, что метод геометрической регрессии не подходит для этого набора данных.